# House price prediciton regression - TFX

In [1]:
import os
import pathlib
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

/home/vagrant/anaconda3/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '


In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.1.0
TFX version: 0.21.2


## Path initialization

In [3]:
# This is the root directory for your TFX pip package installation.
_tfx_root = pathlib.Path().parent.absolute().parent

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_housepred_root = _tfx_root.joinpath('tfx_metadata_store')

# This is the path where your model will be pushed for serving.
_serving_model_dir = _housepred_root.joinpath('housepred_model')

# Path to the data storage
_data_root = _tfx_root.joinpath('data')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

The data is coming from the [Melbourne house price prediction Kaggle](https://www.kaggle.com/anthonypino/melbourne-housing-market#Melbourne_housing_FULL.csv)

In [4]:
RAWDATA_PATH = 'https://raw.githubusercontent.com/fabiansd/AI-workshop/master/data/melb_data.csv'

_rawdata_filepath = _data_root.joinpath('melbourne.csv')

urllib.request.urlretrieve(RAWDATA_PATH, _rawdata_filepath)

(PosixPath('/home/vagrant/Documents/Gitfolder/TFX-workshop/data/melbourne.csv'),
 <http.client.HTTPMessage at 0x7ff2154712d0>)

In [5]:
!head {_rawdata_filepath}

Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,,,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,,,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
Abbotsford,129 Charles St,2,h,941000.0,S,Jellis,7/05/2016,2.5,3067.0,2.0,1

In [6]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

## ExampleGen

This component takes the input data path to your data source

In [7]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data /home/vagrant/Documents/Gitfolder/TFX-workshop/data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/CsvExampleGen/examples/1, id: 2)]
        ))

This component produces training and evaluation examples

In [8]:
%%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/CsvExampleGen/examples/1
This cell will be skipped during export to pipeline.


Here we can see the first 3 training examples

In [9]:
%%skip_for_export

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Address"
    value {
      bytes_list {
        value: "25 Bloomburg St"
      }
    }
  }
  feature {
    key: "Bathroom"
    value {
      float_list {
        value: 1.0
      }
    }
  }
  feature {
    key: "Bedroom2"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "BuildingArea"
    value {
      float_list {
        value: 79.0
      }
    }
  }
  feature {
    key: "Car"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "CouncilArea"
    value {
      bytes_list {
        value: "Yarra"
      }
    }
  }
  feature {
    key: "Date"
    value {
      bytes_list {
        value: "4/02/2016"
      }
    }
  }
  feature {
    key: "Distance"
    value {
      float_list {
        value: 2.5
      }
    }
  }
  feature {
    key: "Landsize"
    value {
      float_list {
        value: 156.0
      }
    }
  }
  feature {
    key: "Lattitude"
    value {
      float_list {

## StatisticsGen

This comp cpmputes statistics over yout dataset for data analysis. It uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library. After it has been run we can display the statistics.

In [10]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/StatisticsGen/statistics/2/eval.
/home/vagrant/anaconda3/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(type_name: ExampleStatistics, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/StatisticsGen/statistics/2, id: 3)]
        ))

In [11]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This cell will be skipped during export to pipeline.


## SchemaGen

Creates a schema based on your data statistics. It takes the statistics generated by StatisticsGen as input looking at the training split by default (which is train: 2/3 and eval: 1/3 by default)

In [12]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(type_name: Schema, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/SchemaGen/schema/3, id: 4)]
        ))

SchemaGen displays all your feature in a column alongside its properties and the domain of categorical values. Learn more about schemas in the [SchemaGen doc](https://www.tensorflow.org/tfx/guide/schemagen)

In [13]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Address',BYTES,required,single,-
'Bathroom',FLOAT,required,single,-
'Bedroom2',FLOAT,required,single,-
'BuildingArea',FLOAT,optional,single,-
'Car',FLOAT,optional,single,-
'CouncilArea',STRING,optional,single,'CouncilArea'
'Date',STRING,required,single,'Date'
'Distance',FLOAT,required,single,-
'Landsize',FLOAT,required,single,-


,Values
Domain,
'CouncilArea',"'Banyule', 'Bayside', 'Boroondara', 'Brimbank', 'Cardinia', 'Casey', 'Darebin', 'Frankston', 'Glen Eira', 'Greater Dandenong', 'Hobsons Bay', 'Hume', 'Kingston', 'Knox', 'Macedon Ranges', 'Manningham', 'Maribyrnong', 'Maroondah', 'Melbourne', 'Melton', 'Monash', 'Moonee Valley', 'Moreland', 'Nillumbik', 'Port Phillip', 'Stonnington', 'Unavailable', 'Whitehorse', 'Whittlesea', 'Wyndham', 'Yarra', 'Yarra Ranges'"
'Date',"'1/07/2017', '10/09/2016', '10/12/2016', '11/02/2017', '11/03/2017', '12/06/2016', '12/08/2017', '12/11/2016', '13/05/2017', '13/08/2016', '14/05/2016', '15/07/2017', '15/10/2016', '16/04/2016', '16/07/2016', '16/09/2017', '17/06/2017', '17/09/2016', '18/03/2017', '18/06/2016', '19/08/2017', '19/11/2016', '20/05/2017', '22/04/2017', '22/05/2016', '22/07/2017', '22/08/2016', '23/04/2016', '23/09/2017', '24/06/2017', '24/09/2016', '25/02/2017', '26/07/2016', '26/08/2017', '27/05/2017', '27/06/2016', '27/11/2016', '28/01/2016', '28/05/2016', '28/08/2016', '29/04/2017', '29/07/2017', '3/06/2017', '3/09/2016', '3/09/2017', '3/12/2016', '30/07/2016', '4/02/2016', '4/03/2017', '4/06/2016', '6/05/2017', '6/08/2016', '7/05/2016', '7/11/2016', '8/04/2017', '8/07/2017', '8/10/2016', '9/09/2017'"
'Method',"'PI', 'S', 'SA', 'SP', 'VB'"
'Regionname',"'Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan', 'Northern Victoria', 'South-Eastern Metropolitan', 'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'"
'Type',"'h', 't', 'u'"


This cell will be skipped during export to pipeline.


## ExampleValidator

Detects anomalies in the data, based on the expectations defined by the schema. The ExampleValidator component will input the statistics from SchemaGen and the schema from SchemaGen.

By default, it compares the statistics from the evaluation split to the schema from the training split.

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(type_name: ExampleAnomalies, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/ExampleValidator/anomalies/4, id: 5)]
        ))

We can then visualize anomalies as a table

In [15]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'CouncilArea',Unexpected string values,Examples contain values missing from the schema: Moorabool (<1%).


This cell will be skipped during export to pipeline.


## Transform

Performs feature engineering for both training and serving. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started).

Transform takes the data from ExampleGen, the schema from SchemaGen and a python module that contains user-defined Transform code.

First, we define a few constants for feature engineering in the prediction_constants.py file. Note: The %%writefile cell magic will save the contents of the cell as a .py file on disk. This allows the Transform component to load your code as a module.

In [16]:
_melbourne_constants_module_file = 'melbourne_constants.py'

In [17]:
%%skip_for_export
%%writefile {_melbourne_constants_module_file}

MAX_CATEGORICAL_FEATURE_VALUES = [303, 3, 5, 10, 32, 8]

CATEGORICAL_FEATURE_KEYS = [
    'Suburb',
    'Address',
    'Type',
    'Method',
    'SellerG', 
    'Date',
    'CouncilArea',
    'Regionname'
]

# Numerical features
DENSE_FLOAT_FEATURE_KEYS = ['Rooms',
                            'Price',
                            'Distance',
                            'Postcode',
                            'Bedroom2',
                            'Bathroom',
                            'Car']
#                            'Landsize',
#                            'BuildingArea',
#                            'YearBuilt']

# Number of buckets used by tf.transform for encoding numerical features into a bucket generaliation
FEATURE_BUCKET_COUNT = 100

BUCKET_FEATURE_KEYS = []
#    'Lattitude', 
#    'Longtitude'
#]

Overwriting melbourne_constants.py
This cell will be skipped during export to pipeline.


We then define a data preprocessing script as a python file for the Transform module

In [18]:
_melbourne_transform_module_file = 'melbourne_transform.py'

In [19]:
%%skip_for_export
%%writefile {_melbourne_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import melbourne_constants

_DENSE_FLOAT_FEATURE_KEYS = melbourne_constants.DENSE_FLOAT_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = melbourne_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = melbourne_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = melbourne_constants.CATEGORICAL_FEATURE_KEYS


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}
    
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        outputs[key] = tft.scale_to_z_score(inputs[key])

    for key in _BUCKET_FEATURE_KEYS:
        outputs[key] = tft.bucketize(inputs[key], _FEATURE_BUCKET_COUNT, always_return_num_quantiles=False)
        
    for key in _CATEGORICAL_FEATURE_KEYS:
        outputs[key] = tft.compute_and_apply_vocabulary(inputs[key])

#     for key in _CATEGORICAL_FEATURE_KEYS:
#         outputs[_transformed_name(key)] = _fill_in_missing(inputs[key])

    return outputs


Overwriting melbourne_transform.py
This cell will be skipped during export to pipeline.


In [20]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_melbourne_transform_module_file))
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


INFO:absl:Using 1 process(es) for Beam pipeline execution.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/Transform/transform_graph/5/.temp_path/tftransform_tmp/2fa1dc7c39724e43a914ee4e87604f51/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/Transform/transformed_examples/5, id: 7)]
        ))

Finally we can see the result from running Transform. This component produces two types of outputs:

 - transform_graph: is the graph that can perform preprocessing operations (included in serving and evaluation)
 - Transformed_examples: the preprcessed training and eval data
 

In [21]:
transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/Transform/transform_graph/5, id: 6)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-25T17_48_25.223118-0i2u253o/Transform/transformed_examples/5, id: 7)]
)}

This is the directory containing the artifiacts in three subdirectories

In [22]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'metadata', 'transform_fn']

The `transformed_metadata` subdirectory contains the schema of the preprocessed data. The `transform_fn` subdirectory contains the actual preprocessing graph. The `metadata` subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [23]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Address"
    value {
      int64_list {
        value: 5036
      }
    }
  }
  feature {
    key: "Bathroom"
    value {
      float_list {
        value: -0.7645469903945923
      }
    }
  }
  feature {
    key: "Bedroom2"
    value {
      float_list {
        value: -0.947419285774231
      }
    }
  }
  feature {
    key: "Car"
    value {
      float_list {
        value: -1.6657211780548096
      }
    }
  }
  feature {
    key: "CouncilArea"
    value {
      int64_list {
        value: 7
      }
    }
  }
  feature {
    key: "Date"
    value {
      int64_list {
        value: 56
      }
    }
  }
  feature {
    key: "Distance"
    value {
      float_list {
        value: -1.3039743900299072
      }
    }
  }
  feature {
    key: "Method"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Postcode"
    value {
      float_list {
        value: -0.4180959463119507
      }
    }
  }
  feature {
    key: "P

## Trainer

The trainer trains a model either defines in the Estimator API og the Keras API using the [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md). 

Trainer takes the Schema from SchemaGen, the transformed data, the graph from Transform, training parameters and a model containing customized model code.

In [24]:
_melbourne_trainer_module_file = 'melbourne_trainer.py'

In [25]:
%%skip_for_export
%%writefile {_melbourne_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

from tfx.components.trainer.executor import TrainerFnArgs

import melbourne_constants

_DENSE_FLOAT_FEATURE_KEYS = melbourne_constants.DENSE_FLOAT_FEATURE_KEYS
_FEATURE_BUCKET_COUNT = melbourne_constants.FEATURE_BUCKET_COUNT
_BUCKET_FEATURE_KEYS = melbourne_constants.BUCKET_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = melbourne_constants.CATEGORICAL_FEATURE_KEYS
_MAX_CATEGORICAL_FEATURE_VALUES = melbourne_constants.MAX_CATEGORICAL_FEATURE_VALUES


def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(
        filenames,
        compression_type='GZIP')


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        #feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)
        #transformed_features.pop(_transformed_name(_LABEL_KEY))

        return model(transformed_features)

    return serve_tf_examples_fn


def _input_fn(file_pattern: Text,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
        file_pattern: input tfrecord file pattern.
        tf_transform_output: A TFTransformOutput.
        batch_size: representing the number of consecutive elements of returned
        dataset to combine in a single batch

    Returns:
        A dataset that contains (features, indices) tuple where features is a
        dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn)

    return dataset


def _build_keras_model(hidden_units: List[int] = None) -> tf.keras.Model:
    """Creates a DNN Keras model for classifying taxi data.

    Args:
    hidden_units: [int], the layer sizes of the DNN (input layer first).

    Returns:
    A keras Model.
    """
    real_valued_columns = [
        tf.feature_column.numeric_column(key, shape=())
        for key in _DENSE_FLOAT_FEATURE_KEYS
    ]
    categorical_columns = [
        tf.feature_column.categorical_column_with_identity(
        key, num_buckets=_FEATURE_BUCKET_COUNT, default_value=0)
        for key in _BUCKET_FEATURE_KEYS
    ]
    #categorical_columns += [
    #    tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
    #        key,
    #        num_buckets=num_buckets,
    #        default_value=0) for key, num_buckets in zip(
    #            _MAX_CATEGORICAL_FEATURE_VALUES,
    #            _CATEGORICAL_FEATURE_KEYS)
    #]
    categorical_columns += [
        tf.feature_column.categorical_column_with_identity(  # pylint: disable=g-complex-comprehension
            key,
            num_buckets=50,
            default_value=0) for key in _CATEGORICAL_FEATURE_KEYS
    ]
    indicator_column = [
        tf.feature_column.indicator_column(categorical_column)
        for categorical_column in categorical_columns
    ]

    model = _wide_and_deep_classifier(
        # TODO(b/139668410) replace with premade wide_and_deep keras model
        wide_columns=indicator_column,
        deep_columns=real_valued_columns,
        dnn_hidden_units=hidden_units or [100, 70, 50, 25])
    return model


def _wide_and_deep_classifier(wide_columns, deep_columns, dnn_hidden_units):
    """Build a simple keras wide and deep model.

    Args:
        wide_columns: Feature columns wrapped in indicator_column for wide (linear)
        part of the model.
        deep_columns: Feature columns for deep part of the model.
        dnn_hidden_units: [int], the layer sizes of the hidden DNN.

    Returns:
        A Wide and Deep Keras model
    """
  # Following values are hard coded for simplicity in this example,
  # However prefarably they should be passsed in as hparams.

  # Keras needs the feature definitions at compile time.
  # TODO(b/139081439): Automate generation of input layers from FeatureColumn.
    input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _DENSE_FLOAT_FEATURE_KEYS
    }
  #input_layers.update({
  #    colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
  #    for colname in _transformed_names(_VOCAB_FEATURE_KEYS)
  #})
    input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _BUCKET_FEATURE_KEYS
    })
    input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _CATEGORICAL_FEATURE_KEYS
    })

  # TODO(b/144500510): SparseFeatures for feature columns + Keras.
    deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes)(deep)
    wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

    output = tf.keras.layers.Dense(
      1, activation='sigmoid')(
          tf.keras.layers.concatenate([deep, wide]))

    model = tf.keras.Model(input_layers, output)
    model.compile(
      loss='binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(lr=0.001),
      metrics=[tf.keras.metrics.BinaryAccuracy()])
    model.summary(print_fn=absl.logging.info)
    return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
    """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  # Number of nodes in the first layer of the DNN
    first_dnn_layer_size = 100
    num_dnn_layers = 4
    dnn_decay_factor = 0.7

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, tf_transform_output, 40)
    eval_dataset = _input_fn(fn_args.eval_files, tf_transform_output, 40)

    model = _build_keras_model(
      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(first_dnn_layer_size * dnn_decay_factor**i))
          for i in range(num_dnn_layers)
      ])

  # This log path might change in the future.
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir, update_freq='batch')
    model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting melbourne_trainer.py
This cell will be skipped during export to pipeline.


This training code is passed into the TFX Trainer comonent.

In [26]:
trainer = Trainer(
    module_file=os.path.abspath(_melbourne_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))

context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.


Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


INFO:absl:Model: "model"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Address (InputLayer)            [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Bathroom (InputLayer)           [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Bedroom2 (InputLayer)           [(None,)]            0                                            
INFO:absl:_____________________________________________________________________________________________

INFO:absl:__________________________________________________________________________________________________
INFO:absl:dense_4 (Dense)                 (None, 1)            435         concatenate[0][0]                
INFO:absl:==================================================================================================
INFO:absl:Total params: 13,379
INFO:absl:Trainable params: 13,379
INFO:absl:Non-trainable params: 0
INFO:absl:__________________________________________________________________________________________________


ValueError: The two structures don't have the same nested structure.

First structure: type=SparseTensorSpec str=SparseTensorSpec(TensorShape([40, None]), tf.int64)

Second structure: type=Tensor str=Tensor("Address:0", shape=(None,), dtype=int32)

More specifically: Substructure "type=SparseTensorSpec str=SparseTensorSpec(TensorShape([40, None]), tf.int64)" is a sequence, while substructure "type=Tensor str=Tensor("Address:0", shape=(None,), dtype=int32)" is not
Entire first structure:
.
Entire second structure:
.

## Tensorboard: training analysis

The Trainer artifact is a directory containing the model subdirectories

In [ ]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

We can connect